# Overlapping or inside?

This notebook demonstrates how to determine if a sticky note overlaps or is inside of a circle shape.

The sample mural for this notebook is here: [Sample mural](https://github.com/spackows/MURAL-API-Samples/blob/main/murals/sample-03_overlap-inside-circle.json)

You can create that mural using the notebook here: [Mural creating notebook](https://github.com/spackows/MURAL-API-Samples/blob/main/notebooks/sample-01_create-a-mural.ipynb)

The sample mural for this notebook looks like the following image:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-03_overlap-inside-circle.png" width="50%" title="Image of a mural" />

## Step 1: Collect the mural ID

You can find the mural ID in the url of a mural.

Mural urls look something like this:

```
https://app.mural.co/t/<workspace>/m/<workspace>/<id>/...
```

What you need to pass to the MURAL API is just after the `/m/`: the \<workspace> and the \<id>.  And you need to join then with a period.

For example, if you have a mural with this url:

```
https://app.mural.co/t/teamideas1234/m/teamideas1234/1234567890123/...
```

Then, the mural ID is: `teamideas1234.1234567890123`

In [1]:
g_mural_id = ""

## Step 2: Collect your OAuth token

In [2]:
g_auth_token = ""

## Step 3: Read the widgets from the mural

In [5]:
import requests
import json

def listWidgets( mural_id, auth_token ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets"
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    return response_json["value"]

In [6]:
widgets_arr = listWidgets( g_mural_id, g_auth_token )
print( json.dumps( widgets_arr, indent = 2 ) )

[
  {
    "contentEditedBy": {
      "firstName": "Sarah",
      "lastName": "Packowski",
      "type": "member"
    },
    "contentEditedOn": 1651017000770,
    "createdBy": {
      "firstName": "Sarah",
      "lastName": "Packowski",
      "type": "member"
    },
    "createdOn": 1651017000770,
    "height": 138,
    "hidden": false,
    "hideEditor": false,
    "hideOwner": false,
    "id": "08236fbb-198e-4c25-b6b9-7df30ed73589",
    "instruction": "",
    "locked": false,
    "lockedByFacilitator": false,
    "parentId": null,
    "presentationIndex": -1,
    "rotation": 0,
    "stackingOrder": 1,
    "title": "",
    "updatedBy": {
      "firstName": "Sarah",
      "lastName": "Packowski",
      "type": "member"
    },
    "updatedOn": 1651017000771,
    "width": 138,
    "x": 233.94,
    "y": 156.77999999999997,
    "minLines": 6,
    "shape": "rectangle",
    "style": {
      "backgroundColor": "#CB8CF9FF",
      "bold": false,
      "font": "proxima-nova",
      "fontSize": 23,

## Step 4: Find the stickies and the circle

*You can pass an optional parameter to the `/getmuralwidgets` endpoint to collect only widgets of a certain type, but because our mural is so simple, it just as easy to call the endpoint once and then post-process the full list of widgets.

In [7]:
def getStickies( widgets_arr ):
    stickies_arr = []
    for widget in widgets_arr:
        if ( "type" in widget ) and ( "sticky note" == widget["type"] ):
            stickies_arr.append( { "id"     : widget["id"],
                                   "x"      : widget["x"],
                                   "y"      : widget["y"],
                                   "width"  : widget["width"],
                                   "height" : widget["height"],
                                   "text"   : widget["text"]
                                 } )
    return stickies_arr

def getShapes( widgets_arr ):
    shapes_arr = []
    for widget in widgets_arr:
        if ( "type" in widget ) and ( "shape" == widget["type"] ):
            shapes_arr.append( { "id"     : widget["id"],
                                 "x"      : widget["x"],
                                 "y"      : widget["y"],
                                 "width"  : widget["width"],
                                 "height" : widget["height"]
                                } )
    return shapes_arr

In [8]:
stickies_arr = getStickies( widgets_arr )
print( json.dumps( stickies_arr, indent = 2 ) )

[
  {
    "id": "08236fbb-198e-4c25-b6b9-7df30ed73589",
    "x": 233.94,
    "y": 156.77999999999997,
    "width": 138,
    "height": 138,
    "text": "This is outside"
  },
  {
    "id": "6956c909-f5bd-45ea-9a84-9dd12a4b0980",
    "x": 564.07,
    "y": 239.28,
    "width": 138,
    "height": 138,
    "text": "This overlaps"
  },
  {
    "id": "8b172423-4cfd-4f5e-87b0-c1492c642178",
    "x": 406.07000000000005,
    "y": 397.28,
    "width": 138,
    "height": 138,
    "text": "This is inside"
  }
]


In [9]:
shapes_arr = getShapes( widgets_arr )
print( json.dumps( shapes_arr, indent = 2 ) )

[
  {
    "id": "0-1651018696848",
    "x": 302.94,
    "y": 314.78,
    "width": 382.750001220703,
    "height": 382.750001220703
  }
]


## Step 5: Figure out if the stickies are overlapping or inside of the circle

In [14]:
def stickyInsideOverlapsCircle( sticky, circle ):
    circle_radius = circle["width"] / 2
    circle_center_x = circle["x"] + circle_radius
    circle_center_y = circle["y"] + circle_radius
    #
    top_left_corner_overlaps = False
    if ( abs( sticky["x"] - circle_center_x ) <= circle_radius ) and \
       ( abs( sticky["y"] - circle_center_y ) <= circle_radius ):
        top_left_corner_overlaps = True
    #
    top_right_corner_overlaps = False
    if ( abs( ( sticky["x"] + sticky["width"] ) - circle_center_x ) <= circle_radius ) and \
       ( abs( sticky["y"] - circle_center_y ) < circle_radius ):
        top_right_corner_overlaps = True
    #
    bottom_left_corner_overlaps = False
    if ( abs( sticky["x"] - circle_center_x ) <= circle_radius ) and \
       ( abs( ( sticky["y"] + sticky["height"] ) - circle_center_y ) <= circle_radius ):
        bottom_left_corner_overlaps = True
    #
    bottom_right_corner_overlaps = False
    if ( abs( ( sticky["x"] + sticky["width"] ) - circle_center_x ) <= circle_radius ) and \
       ( abs( ( sticky["y"] + sticky["height"] ) - circle_center_y ) < circle_radius ):
        bottom_right_corner_overlaps = True
        
    
    if top_left_corner_overlaps and top_right_corner_overlaps and bottom_left_corner_overlaps and bottom_right_corner_overlaps:
        return "inside"
    
    if top_left_corner_overlaps or top_right_corner_overlaps or bottom_left_corner_overlaps or bottom_right_corner_overlaps:
        return "overlaps"
    
    return "outside"

In [15]:
circle = shapes_arr[0];

for sticky in stickies_arr:
    txt = sticky["text"]
    position = stickyInsideOverlapsCircle( sticky, circle )
    print( "[ " + position.ljust(8) + " ]  Sticky text: '" + txt + "'\n" )

[ outside  ]  Sticky text: 'This is outside'

[ overlaps ]  Sticky text: 'This overlaps'

[ inside   ]  Sticky text: 'This is inside'

